In [41]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [3]:
facilities = pd.read_csv("Resources/health-facilities-gh.csv")
tiers = pd.read_csv("Resources/health-facility-tiers.csv")

print(facilities.head())
print()
print("---------------------------------------------------------------------")
print()
print(tiers.head())

    Region          District             FacilityName           Type  \
0  Ashanti     Offinso North        A.M.E Zion Clinic         Clinic   
1  Ashanti  Bekwai Municipal        Abenkyiman Clinic         Clinic   
2  Ashanti      Adansi North     Aboabo Health Centre  Health Centre   
3  Ashanti     Afigya-Kwabre  Aboabogya Health Centre  Health Centre   
4  Ashanti            Kwabre     Aboaso Health Centre  Health Centre   

           Town   Ownership  Latitude  Longitude  
0      Afrancho        CHAG   7.40801   -1.96317  
1  Anwiankwanta     Private   6.46312   -1.58592  
2   Aboabo No 2  Government   6.22393   -1.34982  
3     Aboabogya  Government   6.84177   -1.61098  
4        Aboaso  Government   6.84177   -1.61098  

---------------------------------------------------------------------

    Region                   Facility  Tier
0  Ashanti    PEACE AND LOVE HOSPITAL     2
1  Ashanti             TODAH HOSPITAL     2
2  Ashanti  ASOKWA CHILREN'S HOSPITAL     2
3  Ashanti   

In [4]:
# Data clean. Create 3 base tables: "Regions", "Districts", "Facilities"
# There are failities that are not classified in tiers.

# Standarize column names
tiers.columns = ["Region", "FacilityName", "Tier"]

# Facility name to uppercase in the Facilities table, to match the format in the Tiers dataframe
facilities.FacilityName = facilities.FacilityName.str.upper()

In [5]:
# Find out how many facilities in Ghana are not categorized by tiers

f = pd.merge(facilities, tiers, on="FacilityName", how="left")
f.count()

Region_x        3768
District        3768
FacilityName    3768
Type            3768
Town            3615
Ownership       3768
Latitude        3744
Longitude       3744
Region_y         229
Tier             229
dtype: int64

In [ ]:
# First conclusion: Most healthcare facilities in Ghana are not categorized by tiers, therefore, it is difficult
# to determin the healthcare capacity of a district

# Second: There are facilities mentioned in the Tier table, but do not exist in the Facilities table.

In [17]:
# Get the Region table
regions = pd.DataFrame( facilities["Region"].unique() )
regions = regions.reset_index()
regions.columns = ["id", "region"]
regions

,id,region
0,0,Ashanti
1,1,Brong Ahafo
2,2,Central
3,3,Eastern
4,4,Greater Accra
5,5,Northern
6,6,Upper East
7,7,Upper West
8,8,Volta
9,9,Western


In [18]:
# Store it in Postgress
rds_connection_string = "postgres:Talegazo0931@localhost:5432/GhanaHealthcare"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['regions', 'districts']

In [19]:
regions.to_sql(name='regions', con=engine, if_exists='append', index=False)
regionsId = pd.read_sql_query('select * from regions', con=engine)
regionsId.head()

,id,region
0,0,Ashanti
1,1,Brong Ahafo
2,2,Central
3,3,Eastern
4,4,Greater Accra


In [55]:
facilitiesId = pd.merge(facilities, regionsId, left_on="Region", right_on="region", how="inner")
facilitiesId.head(6)

,Region,District,FacilityName,Type,Town,Ownership,Latitude,Longitude,id,region
0,Ashanti,Offinso North,A.M.E ZION CLINIC,Clinic,Afrancho,CHAG,7.40801,-1.96317,0,Ashanti
1,Ashanti,Bekwai Municipal,ABENKYIMAN CLINIC,Clinic,Anwiankwanta,Private,6.46312,-1.58592,0,Ashanti
2,Ashanti,Adansi North,ABOABO HEALTH CENTRE,Health Centre,Aboabo No 2,Government,6.22393,-1.34982,0,Ashanti
3,Ashanti,Afigya-Kwabre,ABOABOGYA HEALTH CENTRE,Health Centre,Aboabogya,Government,6.84177,-1.61098,0,Ashanti
4,Ashanti,Kwabre,ABOASO HEALTH CENTRE,Health Centre,Aboaso,Government,6.84177,-1.61098,0,Ashanti
5,Ashanti,Offinso North,ABOFFOUR HEALTH CENTRE,Health Centre,Aboffour,Government,7.12986,-1.73294,0,Ashanti


In [57]:
# Create the unique districts table and store them in the database
districts = facilitiesId[["District", "id"]]
districts = districtId.drop_duplicates()
districts = districts.reset_index()
districts.columns= ["id", "district", "region_id"]
districts

,id,district,region_id
0,0,Offinso North,0
1,1,Bekwai Municipal,0
2,2,Adansi North,0
3,3,Afigya-Kwabre,0
4,4,Kwabre,0
5,6,Atwima-Nwabiagya,0
6,7,Atwima-Kwanwoma,0
7,8,Ejisu-Juaben Municipal,0
8,9,Mampong Municipal,0
9,10,Amansie West,0


In [52]:
districts.to_sql(name='districts', con=engine, if_exists='replace', index=False)
districtsId = pd.read_sql_query('select * from districts', con=engine)
districtsId.head()

,id,district,region_id
0,0,Offinso North,0
1,1,Bekwai Municipal,0
2,2,Adansi North,0
3,3,Afigya-Kwabre,0
4,4,Kwabre,0


In [85]:
# Get the Facilities and store them in the database
namesId = pd.merge(facilitiesId, districts, left_on="District", right_on="district", how="inner")
namesId = namesId[["FacilityName", "Type", "Ownership", "Latitude", "Longitude", "id_y"]]
namesId.columns = ["facility", "type", "ownership", "latitude", "longitude", "district_id"]
namesId = namesId.reset_index()
namesId = pd.merge(namesId, tiers, left_on="facility", right_on="FacilityName", how="left")
namesId = namesId[["facility", "type", "ownership", "latitude", "longitude", "district_id", "Tier"]]
namesId = namesId.rename(columns={"Tier":"tier"})
namesId = namesId.fillna(0)
namesId = namesId.reset_index()
namesId = namesId.rename(columns={"index":"id"})
namesId

,id,facility,type,ownership,latitude,longitude,district_id,tier
0,0,A.M.E ZION CLINIC,Clinic,CHAG,7.408010,-1.963170,0,0.0
1,1,ABOFFOUR HEALTH CENTRE,Health Centre,Government,7.129860,-1.732940,0,0.0
2,2,AKOMADAN HEALTH CENTRE,Health Centre,Government,7.405810,-1.960720,0,0.0
3,3,ANYINASUSU SDA CENTRE,Clinic,CHAG,7.014840,-1.708740,0,0.0
4,4,DORCAS MEMORIAL CLINIC,Clinic,CHAG,7.396940,-1.952970,0,0.0
5,5,NKENKASU DISTRICT HOSPITAL,District Hospital,Government,7.319570,-1.902890,0,0.0
6,6,NYAMEBEKYERE CLIINIC,Clinic,Private,6.597212,-1.578999,0,0.0
7,7,ABENKYIMAN CLINIC,Clinic,Private,6.463120,-1.585920,1,3.0
8,8,AHMADIYAA MISSION HOSPITAL,Hospital,Private,6.509860,-1.536490,1,0.0
9,9,AHWENE MEMORIAL HOSPITAL,Hospital,Private,6.454910,-1.570710,1,0.0


In [86]:
namesId.to_sql(name='facilities', con=engine, if_exists='replace', index=False)
names = pd.read_sql_query('select * from facilities', con=engine)
names.head()

,id,facility,type,ownership,latitude,longitude,district_id,tier
0,0,A.M.E ZION CLINIC,Clinic,CHAG,7.40801,-1.96317,0,0.0
1,1,ABOFFOUR HEALTH CENTRE,Health Centre,Government,7.12986,-1.73294,0,0.0
2,2,AKOMADAN HEALTH CENTRE,Health Centre,Government,7.40581,-1.96072,0,0.0
3,3,ANYINASUSU SDA CENTRE,Clinic,CHAG,7.01484,-1.70874,0,0.0
4,4,DORCAS MEMORIAL CLINIC,Clinic,CHAG,7.39694,-1.95297,0,0.0
